In [1]:
import builtwith
import whois
import requests
from bs4 import BeautifulSoup

In [2]:
URL='https://www.meneame.net'

In [3]:
USER_AGENT = "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86.0) Gecko/20100101 Firefox/86.0"
headers = {"user-agent": USER_AGENT}
page=requests.get(URL, headers=headers)

In [4]:
import sys

meneame = page.content
soup = BeautifulSoup(meneame,"html.parser")
#TRAIEM EL SOUP PRETTY A UN FITXER PER FER UN COP D'ULL
meneame_pretty = open("meneame_pretty.html", "w")
print(soup.prettify(), file=meneame_pretty)
meneame_pretty.close()

In [5]:
#Titular
#Web
for i in soup.find_all('div',{'class':'center-content'}):
# mirar tota la info que surt aquí
    noticia=i.find('a')
    titular = noticia.string
    web = noticia['href']
    print(titular)
    print(web)

 Los bancos cobran hasta 240 euros anuales en comisiones a los pensionistas más vulnerables 
https://www.elsaltodiario.com/pensiones/hasta-240-euros-anuales-quedan-bancos-comisiones-pensionistas-mas-vulnerables
 El Supremo confirma la multa más alta de la historia de RTVE por publicidad encubierta 
https://www.elconfidencial.com/empresas/2021-03-25/supremo-confirma-multa-rtve-publicidad_3004947/
 La camiseta con la que Noruega carga contra la FIFA por el Mundial y que aplauden en redes 
https://as.com/futbol/2021/03/24/videos/1616621279_582949.html
 Carreras de cerditos. Temporada cuatro. Primera carrera 
https://www.youtube.com/watch?v=-e_J0gLQs-c
 El mayor estudio hasta la fecha confirma la “singularidad genética” de los vascos 
https://elpais.com/ciencia/2021-03-25/el-mayor-estudio-hasta-la-fecha-confirma-la-singularidad-genetica-de-los-vascos.html?prm=ep-app-cabecera
 Más Madrid denuncia que el gasto de EMT en directivos se dispara un 34% mientras se recorta plantilla 
https://masm

In [6]:
#Entrada
for i in soup.find_all('div',{'class':'news-content'}):
#S'ha de veure com relacionar totes les dades
    entrada=(i.string)
    print(entrada)
    print('')

Además del negocio de los fondos privados de pensiones, los grandes bancos obtienen cientos de millones de euros en comisiones a los pensionistas. Los más perjudicados: los que menos cobran.

El recurso del ente público ha sido inadmitido por la Sala de lo Contencioso-Administrativo y tendrá que pagar los casi 730.000 euros de la sanción que le impusieron hace casi dos años.

Odegaard, Haaland y el resto de jugadores saltaron al campo con una camiseta en la que se veía un mensaje dirigido a la FIFA por el Mundial de Catar: "Derechos humanos dentro y fuera del campo".

El canal League of Pigs lanza, periódicamente, carreras de cerditos. Los cerditos, cada cual con su ficha técnica, origen y hándicap específico en función de altura y peso, han de sortear una serie de obstáculos hasta alcanzar la meta, en una trepidante carrera. Los vÍdeos, con una edición muy trabajada, dan resúmenes de las carreras y explicaciones del perfil de cada cerdito corredor.

La investigación ratifica una histo

In [7]:
#Meneos
for i in soup.find_all('div',{'class':'votes'}):
    noticia_m = i.find('a')
    # aixó és perque hi han algunes class:votes que no tenen info
    if noticia_m != None:
        print(noticia_m)
        meneos = noticia_m.string
        print(meneos)

<a href="/story/bancos-cobran-hasta-240-euros-anuales-comisiones-pensionistas" id="a-votes-3479584">76</a>
76
<a href="/story/supremo-confirma-multa-mas-alta-historia-rtve-publicidad" id="a-votes-3479471">113</a>
113
<a href="/story/camiseta-noruega-carga-contra-fifa-mundial-aplauden-redes" id="a-votes-3479607">187</a>
187
<a href="/story/carreras-cerditos-temporada-cuatro-primera-carrera" id="a-votes-3479587">89</a>
89
<a href="/story/mayor-estudio-hasta-fecha-confirma-singularidad-genetica-vascos" id="a-votes-3479657">111</a>
111
<a href="/story/mas-madrid-denuncia-gasto-emt-directivos-dispara-34-mientras" id="a-votes-3479603">288</a>
288
<a href="/story/toni-canto-empadrono-madrid-junta-electoral-decidira-puede-ser" id="a-votes-3479454">221</a>
221
<a href="/story/madrid-supera-hospitalizaciones-covid-dos-comunidades-mas-juntas" id="a-votes-3479665">482</a>
482
<a href="/story/martas-adorable-amenaza-coches-alemanes" id="a-votes-3479375">82</a>
82
<a href="/story/bragueta-armar-moda

In [8]:
#Clics
for i in soup.find_all('div',{'class':'clics'}):
    clics = i.find('span').string
    if clics != None:
        print(clics.string)

84
879
2184
1778
1966
287
1079
1185
4982
5298
1238
3000
521
1176
9599
1446
5733
4019
2046
2541
2039
777
3969
1473
2811


In [9]:
#Comentaris
for i in soup.find_all('div',{'class':'news-details-main'}):
    comentaris = i.find('a')['data-comments-number']
    print(comentaris)

15
33
31
19
119
42
94
124
48
38
101
227
182
35
74
149
116
98
86
74
20
105
150
13
20


In [10]:
#Vots positius
#Vots anónims
#Vots negatius
#Sub
#Karma
for i in soup.find_all('div',{'class':'news-details-data-up'}):
    x = i.find_all('span')
    #x es un bs4.element.ResultSet
    print(x[2].string)
    print(x[5].string)
    print(x[8].string)
    print(x[12].string)
    print(x[13].a.string)
    print('')

59
17
0
331
actualidad

75
38
2
363
actualidad

102
85
6
328
actualidad

52
37
19
331
ocio

55
56
3
340
ciencia

139
149
2
334
actualidad

94
127
15
344
politica

238
244
30
425
actualidad

44
38
3
313
tecnología

50
38
0
329
cultura

229
246
37
363
politica

179
211
23
370
actualidad

213
297
11
361
actualidad

89
84
2
380
actualidad

197
242
50
366
ocio

284
330
0
363
actualidad

129
195
30
407
ocio

85
75
0
412
vacúname

335
559
15
374
actualidad

107
100
4
372
actualidad

83
61
3
337
astronomia

236
325
12
334
politica

109
86
5
348
actualidad

37
32
0
339
cultura

156
91
0
338
ciencia



In [11]:
#Nom usuari
